In [2]:
from tqdm.notebook import trange, tqdm
import requests

In [3]:
import pandas as pd
data = pd.read_csv("COD-selection.csv")

/var/folders/gl/2h8nn4ps031g1jysgll17wz00000gn/T/ipykernel_44875/2708168232.py:2: DtypeWarning: Columns (23,24,25,32,44,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("COD-selection.csv")


In [ ]:
!pip install scidownl PyPDF2 transformers torch sentencepiece

In [4]:
import PyPDF2

In [ ]:
!pip install sentencepiece

In [ ]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]

In [5]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

src_text = [
    """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

device = "mps"
tokenizer_news = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail", max_position_embeddings=2048)
tokenizer_xl = PegasusTokenizer.from_pretrained("google/pegasus-large", max_position_embeddings=2048)
model_news = PegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail", max_position_embeddings=2048).to(device)
model_xl = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large", max_position_embeddings=2048).to(device)
batch = tokenizer_news(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model_news.generate(**batch)
tgt_text = tokenizer_news.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

/usr/local/anaconda3/envs/ml/lib/python3.9/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/anaconda3/envs/ml/lib/python3.9/site-packages/transformers/generation_utils.py:2260: UserWarning: The operator 'aten::remainder.Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1663830697895/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  next_tokens = next_tokens % vocab_size


['PG&E scheduled the blackouts in response to forecasts for high winds amid dry conditions.<n>The aim is to reduce the risk of wildfires.']


In [6]:
from scidownl import scihub_download
import os
def get_summary(doi):
    paper = "https://doi.org/"+ doi
    paper_type = "doi"
    out = "./tmp.pdf"
    scihub_download(paper, paper_type=paper_type, out=out)
    pdfFileObj = open('tmp.pdf', 'rb')

    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    
    text = pdfReader.pages[0].extract_text()

    pdfFileObj.close()
    try:
        os.remove("tmp.pdf")
    except:
        return ""
    prompt = " ".join(text.split(" ")[:400])
    result_length = 200
    inputs = tokenizer_news(prompt, return_tensors="pt").to("mps")
    return tokenizer_news.decode(model_news.generate(inputs["input_ids"])[0])

In [14]:
import json

doi_sum = {}
for i,r in tqdm(data.iterrows(), total=data.shape[0]):
    if i % 10 == 0:
        with open('summaries_inorganic.json', 'w') as fp:
            json.dump(doi_sum, fp)
    summ = get_summary(r.doi)
    if summ == "":
        pass
    doi_sum[r.doi] = summ
    break

with open('summaries_inorganic.json', 'w') as fp:
    json.dump(doi_sum, fp)

  0%|          | 0/44334 [00:00<?, ?it/s]

[INFO] | 2023/01/11 21:22:22 | Choose scihub url [0]: http://sci-hub.se
[INFO] | 2023/01/11 21:22:22 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=doi.org/10.1107/S1600536803023699]
[INFO] | 2023/01/11 21:22:22 | -> Response: status_code=200, content_length=7133
[INFO] | 2023/01/11 21:22:22 | * Extracted information: {'url': 'https://moscow.sci-hub.se/523/bd2a67bf1fcd77eab8b103731ab45492/goreshnik2003.pdf', 'title': '1,4,8,11-Tetraazacyclodecane-1,4,8,11-tetraium bis(aquapentafluoroaluminate) dihydrate. Acta Crystallographica Section E Structure Reports Online, 59(11), m1059–m1061'}
[INFO] | 2023/01/11 21:22:23 | ↓ Successfully download the url to: ./tmp.pdf


100% [............................................................................] 219066 / 219066
